# WebScrapping

Selenium est une bibliothèque Python et un outil utilisé pour automatiser les navigateurs Web afin d'effectuer un certain nombre de tâches. L’un d’entre eux est le web-scraping pour extraire des données et des informations utiles qui autrement ne seraient pas disponibles. Voici un guide étape par étape sur la façon d'utiliser Selenium, l'exemple étant l'extraction des données sur les salaires des joueurs NBA du site Web [hoopshype](https://hoopshype.com/salaries/players/)

### Etape 1 : Installation de Selenium et importation des bibliothèques

In [16]:
#!pip install selenium

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import pandas as pd

### Etape 2 : Installer et accéder à Webdriver

Un Webdriver est un ingrédient essentiel de ce processus. C’est ce qui ouvrira automatiquement votre navigateur pour accéder au site Web de votre choix.  
Cette étape est différente selon le navigateur que vous utilisez pour explorer Internet. Il se trouve que j'utilise Google Chrome. Certains disent que Chrome fonctionne mieux avec Selenium, bien qu'il prenne également en charge Internet Explorer, Firefox, Safari et Opera.  
Pour Chrome, vous devez d'abord télécharger le pilote Web sur [chromedriver](https://chromedriver.chromium.org/downloads).  
Il existe plusieurs options de téléchargement différentes en fonction de votre version de Chrome. Pour localiser la version de Chrome dont vous disposez, cliquez sur les 3 points verticaux dans le coin supérieur droit de la fenêtre de votre navigateur, faites défiler vers le bas pour obtenir de l'aide et sélectionnez « À propos de Google Chrome ». Là, vous verrez votre version. 

Vous devez maintenant savoir où vous avez enregistré le téléchargement de votre pilote Web sur votre ordinateur local.  
Le mien vient d'être enregistré dans mon dossier de téléchargements par défaut que je vais déplacer dans ce projet. Vous pouvez maintenant créer une variable de pilote en utilisant le chemin direct de l'emplacement de votre pilote Web téléchargé.

**Il faut autoriser l'ordinateur à ouvrir un logiciel externe. Chez Apple il faut se rendre dans Confidentialité et Sécurité défiler vers le bas et Autoriser l'utilisation du logiciel.**

In [12]:
chromedriver_path = '../chromedriver-mac-x64/chromedriver'

# Set up the ChromeDriver service
service = Service(executable_path=chromedriver_path)

# Set up Chrome options
options = Options()
# Uncomment the following line if you need headless mode
# options.add_argument('--headless') 

# Initialize the WebDriver with service and options
driver = webdriver.Chrome(service=service, options=options)

## Etape 3 : Accéder au site Web via Python

Étape très simple mais très importante. Vous avez besoin de votre code pour ouvrir le site Web que vous essayez de scrapper.

In [13]:
driver.get('https://hoopshype.com/salaries/players/')

Une fois exécuté, cet extrait de code ouvrira le navigateur sur le site Web de votre choix.

## Etape 4 : Localisez les informations spécifiques que vous récupérez

Afin d’extraire les informations que vous souhaitez récupérer, vous devez localiser le XPath de l’élément.  
**Un XPath est une syntaxe utilisée pour rechercher n'importe quel élément sur une page Web.**  
Pour localiser le XPath de l'élément, mettez en surbrillance le premier dans la liste de ce que vous recherchez, faites un clic droit et sélectionnez inspecter ; cela ouvre les outils de développement.  
  
Pour mon exemple, je souhaite d’abord localiser les noms des joueurs NBA, je sélectionne donc d’abord Stephen Curry.

![image](./documentation/image_00.png)

Dans les outils de développement, on voit désormais l’élément « Stephen Curry » apparaître comme tel.

![image2](./documentation/image_01.png)

Cet élément peut facilement être traduit en son XPath, mais d’abord, nous devons nous rappeler que nous n’essayons pas seulement de localiser cet élément, mais tous les noms de joueurs.

Le point commun entre le nom des joueurs est **\<td class="name">**, c'est donc ce que nous utiliserons pour créer une liste de tous les noms de joueurs.  
  
Cela traduit en XPath ressemble à **//td\[@class="name"]**.  
  
En décomposant cela, tous les XPaths sont précédés de la double barre oblique, que nous voulons dans une balise td, chaque classe de cette balise td devant correspondre à « nom ». Nous pouvons maintenant créer la liste des noms de joueurs avec cette fonction Selenium.

In [17]:
players = driver.find_elements(By.XPATH, '//td[@class="name"]')

Et maintenant, pour mettre le texte de chaque nom de joueur dans une liste, nous écrivons cette fonction.

In [18]:
players_list = []
for p in range(len(players)):
	players_list.append(players[p].text)

Lors de l'inspection de ces éléments et de la traduction vers XPath, nous pouvons ignorer le style et la valeur des données, en nous souciant uniquement de la classe.

In [19]:
salaries = driver.find_elements(By.XPATH, '//td[@class="hh-salaries-sorted"]')

Et le texte de la liste des salaires…

In [20]:
salaries_list = []
for s in range(len(salaries)):
	salaries_list.append(salaries[s].text)

## Etape 5 : Appliquez pour chaque année disponible et liez le tout ensemble

Souvent, lorsque vous utilisez Selenium, vous tenterez de récupérer des données situées sur plusieurs pages différentes du même site Web. Dans mon exemple, [hoopshype](hoopshype.com) dispose de données sur les salaires de la NBA remontant à la saison 1990/91.

![image3](./documentation/image_02.png)

![image4](./documentation/image_03.png)

Comme vous pouvez le constater, la différence entre l'URL de chaque saison est simplement une question d'années incluses à la fin. Ainsi, l'URL de la saison 2018/19 est **https://hoopshype.com/salaries/players/2018-2019/** et l'URL de la saison 1990/91 est **https://hoopshype.com/salaries/players/1990- 1991/**. Avec cela, nous pouvons créer une fonction qui parcourt chaque année et accède à chaque URL individuellement, puis regroupe toutes les étapes que nous avons précédemment présentées pour chaque année individuellement. J'associe également chaque joueur à son salaire pour cette saison, je le place dans une trame de données temporaire, j'ajoute l'année à cette trame de données temporaire, puis j'ajoute cette trame de données temporaire à une trame de données principale qui inclut toutes les données que nous avons acquises. Le code final est ci-dessous !

In [23]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

# Crée le DataFrame maître
df = pd.DataFrame(columns=['Player', 'Salary', 'Year'])

chromedriver_path = '../chromedriver-mac-x64/chromedriver'
service = Service(executable_path=chromedriver_path)
options = Options()
driver = webdriver.Chrome(service=service, options=options)

for yr in range(1990, 2019):
    page_num = str(yr) + '-' + str(yr + 1) + '/'
    url = 'https://hoopshype.com/salaries/players/' + page_num
    driver.get(url)

    players = driver.find_elements(By.XPATH, '//td[@class="name"]')
    salaries = driver.find_elements(By.XPATH, '//td[@class="hh-salaries-sorted"]')

    players_list = []
    for p in range(len(players)):
        players_list.append(players[p].text)

    salaries_list = []
    for s in range(len(salaries)):
        salaries_list.append(salaries[s].text)

    # Liste des tuples contenant le nom et le salaire de chaque joueur
    data_tuples = list(zip(players_list[1:], salaries_list[1:]))
    # Crée un DataFrame temporaire pour les données de l'année en cours
    temp_df = pd.DataFrame(data_tuples, columns=['Player', 'Salary'])
    temp_df['Year'] = yr

    df = pd.concat([df, temp_df], ignore_index=True)

driver.close()


In [24]:
print(df.head())

             Player      Salary  Year
0     Patrick Ewing  $4,250,000  1990
1  Hot Rod Williams  $3,785,000  1990
2   Hakeem Olajuwon  $3,175,000  1990
3   Charles Barkley  $2,900,000  1990
4      Chris Mullin  $2,850,000  1990
